# Overview

We will build a Flask web application and learn how to deploy the Imagenet classifier using pytorch's prebuilt densenet121 model. The pre-built model was originally trained to identify 1000 classes. We will also learn how to deploy the web app in Azure cloud infrastructure. Here is a quick peek at the application we will build from scratch.

![](images/preview_ostrich.png "Web app preview")

you can have a look at the deployed app [here](http://52.157.86.185:8000) 
<br>
Here is the steps which we will follow to achieve the end result.
* Build the `html` pages required for UI.
* Make the `html` pages interactive by creating `routes`.
* Build the model.
* Deploy the web app with model using 
    * Azure Virtual Machines.
    * Azure built in web app deployment mechanism using github 'actions'.

## Building  UI:

It is good to have the general idea about how our app should look like. No, we are not talking about the aesthetic of the application. We are talking about how the user is going to interact with the app. Suppose for example in the web app we are going to implement, it is related to image, so we need to provide a button option to the end user to click and upload the image. Once user selects the image, it is nice to have it displayed on the screen. Also need to make sure user selects only image files not some other files like executables or scripts which will compromise our web application from malicious activities. We will touch up on the security aspects of the website later. Let us see how we want our skeleton app to look like.

![](images/skeleton.jpg "skeleton ui")

Below tree diagram shows how our code is organized. We will break it down layer by layer as we make progress.


```sh
Application
├── config.py
├── flask_pytorch_web_app
│   ├── __init__.py
│   ├── model.py
│   ├── static
│   │   ├── css
│   │   │   └── main.css
│   │   ├── data
│   │   │   └── imagenet_class_index.json
│   │   └── images
│   │       └── opencv-logo-1.png
│   ├── templates
│   │   ├── base.html
│   │   ├── predict.html
│   │   └── upload.html
│   └── upload.py
├── README.md
├── setup.py
├── start.sh
└── wsgi.py
```

most of the html and styling aspects of the page lies in the "templates and css" directories.
```sh
│   ├── static
│   │   ├── css
│   │   │   └── main.css
│   ├── templates
│   │   ├── base.html
│   │   ├── predict.html
│   │   └── upload.html
```

## base.html
Here is the complete `base.html`. 

```html
<!DOCTYPE html>
<html lang='en'>
<head>
  {% block head %} {% endblock %}
</head>

<body>
  <div id="header"> 
    <h3> Flask Web App</h3>
  </div>
  <div id="navbar">  
    <ul>
      <li ><a href="{{ url_for('upload') }}">Home</a></li>
    </ul>
  </div>
  <div class="main">
    <div id="container">
      {% block body%}  {% endblock %}
    </div>
  </div>
  <div id="footer"> 
    <img id="logo" src={{ url_for('static', filename='images/opencv-logo-1.png') }} >
  </div>
</body>

</html>
```

The above html code looks complicated at first. But it is extremely simple to understand, if you know few basics. 
<br>
Flask uses a templating syntax called [Jinja](https://jinja.palletsprojects.com/en/2.11.x/templates/#) to render the html. `Jinja2` will greatly simplify `html` page rendering through python code. In its simpletx form, our  `base.html` looks as shown below. 

```html
<!DOCTYPE html>
<html lang='en'>
<head>
  {% block head %} {% endblock %}
</head>

<body>
  {% block body%} {% endblock %}
</body>

</html>
```

Suppose our web application supports multiple pages. We want to have a similar look and feel for all the pages. The content in the body or header may change. But the over all look and feel should remain same. If we try to hand code all of these elements by hand, it becomes a more and more difficult as we keep adding new features and elements in to our website. It soon becomes a maintenance nightmare to update the website. This is where `Jinja` template parser comes to our rescue. It greatly reduces the effort of creating and maintaining our `html` pages. <br>Here `{% block head %} {% endblock %}` and `{% block body%} {% endblock %}` provides a place holder to add or delete  or hide the existing `html tags`. we can use to fill in the required content in new page. `Jinja` templating engine will identifies these blocks and generates the complete html file just when the page is rendered. 
<br>
To understand it even better let us have a look at the `upload.html` file

## upload.html


```html
{% extends 'base.html' %}

{% block head %} 
  <title> Upload Image</title>
{% endblock %}

{% block body %} 
      <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
      {% if obj.is_image_display %}
      <div > 
        <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
      </div>
      {% endif %}
{% endblock %}
```

Here in `upload.html` file we are not writing the complete `html` file, we are just inheriting the `base.html` file using `{% extends 'base.html' %}` since our `base.html` file already has the needed fields.
All you need to do is add or extend the html page according to your needs. `jinja` template engine will do the fill in the blanks for you.



Now in our `update.html` file
```html
{% block head %} 
  <title> Upload Image</title>
{% endblock %}
```
code simply extends the placeholder which we left in `base.html` file in the `head` tag. In this example we are adding a new `title` tag in to our `upload.html` page.

similarly we want a upload button which should prompt the user to select an image file upon clicking the same. That is exactly what we are doing with the below piece of html code. This piece of UI we want it to appear in the body portion of the html file hence we extend the `{% block body%} {% endblock %}` by putting the required html tags to populate the upload button

```html
{% block body%} 
     <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
{% endblock %}
```

To understand the code in between the tag `<form> ..... </form>` tag it is better to look at the final output and compare it back with our code. Here is our page with some css magic, which we will cover shortly.

![](images/upload_button_only.png)

It is html standard to get the user provided data using `form` tags.

```html 
<form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
```

We specify the type of `method` (GET/POST/DELETE/PUT etc) to be used using `method=POST` attribute. In our app we are trying to get the image to be uploaded, hence we need to use `enctype=multipart/form-data` attribute to receive the uploaded image data. `action` attribute is used to pass the image available in the form to a `upload` function in our flask application. The syntax `{{ }}` is the way to tell the `jinja` template engine to find the appropriate method named `upload` in the flask python files. `url_for` is a utility function available in flask library to access the relative path/resources of the flask app. As a best practice, we should always use `url_for` when ever we try to access the resources in our web app, this will help us bind our resources such as logos/css files/images/redirection html files etc to the web app in a secure way.

```html
    <label for="inputfile"> Input image file:  </label>
```
As the tag name `label` suggest it gives a label to our input field to hint the user.

```html
    <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
```
`input` tag attributes are prettymuch self explainatory. The `accept` attributes tells the browser to filter out and display only the files with extensions `.jpg, .jpeg, .png` to be displayed for selection. This will solve the problem of user accidentally selecting the incorrect types of file such as `.txt or .sh` etc.

```html
    <input type="submit" value="Upload">
```
The attribute `submit` will send the message when user clicks the button using the appropriate methods mentioned in `<form method=POST>`. In our case it is `POST` method.
<br>
`value` attribute is the name which is displayed on the `button`.

## predict.html

Similar to `upload.html` we have `predict.html` file. We use this file when we want to do the prediction of the image. Let us look at the `predict.html` file

```html
{% extends 'base.html' %}

{% block head %} 
  <title> Predict Image</title>
{% endblock %}

{% block body %} 
      {% if obj.is_image_display %}
        {% if obj.is_predicted %}
          <div id="prediction"> 
            <h1> Image has : {{ obj.value }}!! </h1> 
          </div>
        {% else %}
          <div>
            <form method=POST enctype=multipart/form-data action="{{ url_for('prediction', filename=obj.image) }}">
              <input type="submit" value="Classify Image" id="blockButton">
            </form>
          </div> 
        {% endif %}
        <div > 
          <img id="inputImage" src={{ url_for('send_file', filename=obj.image) }} alt="User Image">
        </div>
      {% endif %}
{% endblock %}

```

If you look carefully this file looks very similar to `upload.html` we are still using the same inheritance  `{% extends 'base.html' %}` and extending the `base.html` file and fill in the `{% block body %}` and `{% block body %}`. The additional `{% if %} {% else %}  {% endif %}` are standard `jinja` conditionals to enable or disable `div` elements depending on if the image file exists or not. You can read more about the [jinja condtionals syntax here](https://jinja.palletsprojects.com/en/2.11.x/templates/#if).

### CSS

To beuatify the web pages we are using simple `css` file called `main.css`. You can get very creative with CSS with the look and feel of your webpage. There are many tutorials available online to learn and master `css`.
Instead of teaching verything about css, here we just teach you one trick that will help you understand the `css` file and to which specific element of `html` tag `css` styling is being applied.
<br>
If you find attribute as `id` or `class` in html files, you can search for those names in `css` file to find the style applied on that `div`.

for example in our `upload.html` file:

```html
      <div id="form">
        <form method=POST enctype=multipart/form-data action="{{ url_for('upload') }}">
          <label for="inputfile"> Input image file:  </label>
          <input type="file" name="image" id="inputfile" accept=".jpg, .jpeg, .png"> 
          <input type="submit" value="Upload">
        </form>
      </div> 
```

if you search for "form" in our `main.css` file

```css
#form {
  margin: 10px;
  display: block;
  margin-left: auto;
  margin-right: auto;
  width: 50%;
  color: white;
  border: none;
  background-color: #CC7E33;
  padding: 14px 28px;
  font-size: 120%;
  font-weight: bolder;
  cursor: pointer;
  text-align: center;
  border-radius: 5px;
}

```
There you go, this is the `css` style which we have used to in this application for the `form` tag. The "`#`" sign in front of "form" tells us that it is an "`id`" attribute (`<div id="form">`) and if there is a "`.`" it means it is a `class` attribute (`<div class="form">`). To understand the main.css file used in example, this much information is sufficient for now.

We still have not done any linking of page and what action to take on clicking the upload button etc. We will get in to those details shortly. But we have all the UI elements which we wanted. For now we just fast forward a bit to see our results with our initial expectation.
![](images/before_classify_partial.png)